## astropy.wcs
Implements the FITS WCS standard and some commonly used distortion conventions.

This tutorial will show how to create a WCS object from a FITS file and how to use it to transform coordinates.

In [ ]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import os
from astropy.io import fits
from astropy import wcs

Open a file with `astropy.fits` and look at it.

In [ ]:
sip_file_name = os.path.join('data/sip.fits')

sip_file = fits.open(sip_file_name)
sip_file.info()

To create a WCS object pass the header with the WCS kyewords to astropy.wcs.WCS (Primary header in this case).

In [ ]:
w = wcs.WCS(sip_file[0].header)

To perform the transformation from detector to sky, including distortions, pass x and y and an 'origin'. The third argument, 'origin', indicates whether the coordinates are 1-based (like FITS), or 0-based (like python).

The inputs can be numbers, numpy arrays or array like objects.

In [ ]:
ra, dec = w.all_pix2world([1, 1], [2, 2], 1)
print(ra, dec)

Perfom the inverse transformation - from sky to detector coordinates.

If analytical inverse is not available (often the case in the presence of distortion), then an iterative inverse is performed.

In [ ]:
print(w.all_world2pix(ra, dec, 1))

In some cases it is useful to omit the distortion and perform the core WCS transforms only:

In [ ]:
ra, dec = w.wcs_pix2world([1, 1], [2, 2], 1)
print(ra, dec)

In [ ]:
w.wcs_world2pix(ra, dec, 1)

The WCS object can be changed and the new WCS can be written out as a new header.

By default only the primary WCS keywords are written out to the header. Pass a keyword `relax=True` to write out the SIP distortion.

In [ ]:
# The original WCS
w.printwcs()

In [ ]:
w.wcs.crpix = [200, 200]

# Calling *to_header()* without arguments writes
# out the standard WCS keywords.
w.to_header()

# Passing *relax=True* writes out the SIP coefficients
# and all other distortions.
#w.to_header(relax=True)

### TPV distortion

The TPV distortion convention understood by SExtractor and related programs is also understood by `astropy.wcs`.

A header can be read from a text file and used to create the WCS. Here we use a modified header from the Palomar Transient Factory.

In [ ]:
tpv_header = fits.Header.fromtextfile(os.path.join('data', 'PTF_r_chip01_tpv.txt'))

In [ ]:
w_tpv = wcs.WCS(tpv_header)

The keywords starting with `PV` specify the distortion

In [ ]:
w_tpv.to_header()

In the TPV case, `all_pix2world` gives the same result as `wcs_pix2world`.

This is because the underlying WCSLIB library is handling the distortion.

In [ ]:
print(w_tpv.all_pix2world([[500,1000]], 1))

In [ ]:
print(w_tpv.wcs_pix2world([[500,1000]], 1))

### Exercise 1:

- Create a WCS object from the file. 

`dist_file_name = 'data/dist_lookup.fits.gz'`

This file contains all distortions typical for HST imaging data - SIP, lookup_table and det2im (detector to image - correcting detector irregularities). The lookup table and det2im distortions are stored in separate extensions so you will need to pass as a second argument to `wcs.WCS` the file object (already opened with astropy.io.fits).

- Look at the file object with the `info()` method. The lookup_table and det2im distortions are saved in separate extensions.

- Modify one of the WCS keywords and save it to file. (As some of the distortion is saved in extensions, use the method `to_fits()` to save the entire WCS.

### Exercise 2:

The FITS WCS standard supports alternate WCSs in the same eader.
These are defined by the same keywords with a character (A...Z) appended
to them. For example, *CRPIXA1*, etc.

Using the same file create a WCS object for the alternate WCS in this header, by passing also `key='O'` to wcs.WCS().
Compare the two WCSs using the `printwcs()` method`

In [ ]:
dist_file_name = 'data/dist_lookup.fits.gz'
f = fits.open(dist_file_name)